In [2]:


import pandas as pd
from pandas import DataFrame 
df_tennis = pd.read_csv('tennis.csv')
print("\n Given Play Tennis Data Set:\n\n",df_tennis)

df_tennis.keys()[0]



def entropy(probs):  
    import math
    return sum( [-prob*math.log(prob, 2) for prob in probs] )


def entropy_of_list(a_list):  
    from collections import Counter
    cnt = Counter(x for x in a_list)   # Counter calculates the propotion of class
    num_instances = len(a_list)*1.0   # = 14
    print("\n Number of Instances of the Current Sub Class is {0}:".format(num_instances ))
    probs = [x / num_instances for x in cnt.values()]  # x means no of YES/NO
    print("\n Classes:",min(cnt),max(cnt))
    print(" \n Probabilities of Class {0} is {1}:".format(min(cnt),min(probs)))
    print(" \n Probabilities of Class {0} is {1}:".format(max(cnt),max(probs)))
    return entropy(probs) # Call Entropy :
   
   
print("\n  INPUT DATA SET FOR ENTROPY CALCULATION:\n", df_tennis['PlayTennis'])

total_entropy = entropy_of_list(df_tennis['PlayTennis'])
print("\n Total Entropy of PlayTennis Data Set:",total_entropy)

def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    print("Information Gain Calculation of ",split_attribute_name)
    '''
    Takes a DataFrame of attributes, and quantifies the entropy of a target
    attribute after performing a split along the values of another attribute.
    '''
   
    df_split = df.groupby(split_attribute_name)
    for name,group in df_split:
            print("Name:\n",name)
            print("Group:\n",group)
   
    nobs = len(df.index) * 1.0
    print("NOBS",nobs)
    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]
    print([target_attribute_name])
    print(" Entropy List ",entropy_of_list)
    print("DFAGGENT",df_agg_ent)
    df_agg_ent.columns = ['Entropy', 'PropObservations']
    if trace: # helps understand what fxn is doing:
        print(df_agg_ent)
    
    
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy


print('Info-gain for Outlook is :'+str( information_gain(df_tennis, 'Outlook', 'PlayTennis')),"\n")
print('\n Info-gain for Humidity is: ' + str( information_gain(df_tennis, 'Humidity', 'PlayTennis')),"\n")
print('\n Info-gain for Wind is:' + str( information_gain(df_tennis, 'Wind', 'PlayTennis')),"\n")
print('\n Info-gain for Temperature is:' + str( information_gain(df_tennis, 'Temperature','PlayTennis')),"\n")

def id3(df, target_attribute_name, attribute_names, default_class=None):
    
    
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])# class of YES /NO
    
    
    if len(cnt) == 1:
        return next(iter(cnt))  
    elif df.empty or (not attribute_names):
        return default_class  
    else:
        
        default_class = max(cnt.keys())
        
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] #
        index_of_max = gainz.index(max(gainz)) # Index of Best Attribute
        
        best_attr = attribute_names[index_of_max]
        
       
        tree = {best_attr:{}} 
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        
       
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,
                        target_attribute_name,
                        remaining_attribute_names,
                        default_class)
            tree[best_attr][attr_val] = subtree
        return tree

attribute_names = list(df_tennis.columns)
print("List of Attributes:", attribute_names) 
attribute_names.remove('PlayTennis')
print("Predicting Attributes:", attribute_names)

from pprint import pprint
tree = id3(df_tennis,'PlayTennis',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n")

pprint(tree)
attribute = next(iter(tree))
print("Best Attribute :\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())

def classify(instance, tree, default=None): 
    
    print("Instance:",instance)
    attribute = next(iter(tree))    
    print("Attribute:",attribute) 
   
   
    if instance[attribute] in tree[attribute].keys(): 
        result = tree[attribute][instance[attribute]]
        print("Instance Attribute:",instance[attribute],"TreeKeys :",tree[attribute].keys())
        if isinstance(result, dict): 
            return classify(instance, result)
        else:
            return result 
    else:
        return default
df_tennis['predicted'] = df_tennis.apply(classify, axis=1, args=(tree,'No') ) 
    

df_tennis[['PlayTennis', 'predicted']]
training_data = df_tennis.iloc[1:-4] # all but last four instances
test_data  = df_tennis.iloc[-4:] # just the last four
train_tree = id3(training_data, 'PlayTennis', attribute_names)

test_data['predicted2'] = test_data.apply( # <---- test_data source
                                          classify, 
                                          axis=1, 
                                          args=(train_tree,'Yes') ) # <---- train_data tree


print ('\n\n Accuracy is : ' + str( sum(test_data['PlayTennis']==test_data['predicted2'] ) / (1.0*len(test_data.index)) ))


 Given Play Tennis Data Set:

        Sunny   Hot    High    Weak   No
0      Sunny   Hot    High  Strong   No
1   Overcast   Hot    High    Weak  Yes
2       Rain  Mild    High    Weak  Yes
3       Rain  Cool  Normal    Weak  Yes
4       Rain  Cool  Normal  Strong   No
5   Overcast  Cool  Normal  Strong  Yes
6      Sunny  Mild    High    Weak   No
7      Sunny  Cool  Normal    Weak  Yes
8       Rain  Mild  Normal    Weak  Yes
9      Sunny  Mild  Normal  Strong  Yes
10  Overcast  Mild    High  Strong  Yes
11  Overcast   Hot  Normal    Weak  Yes
12      Rain  Mild    High  Strong   No


KeyError: 'PlayTennis'

In [2]:

import pandas as pd
from pandas import DataFrame 
df_tennis = pd.read_csv('tennis.csv')
print("\n Given Play Tennis Data Set:\n\n",df_tennis)
#df_tennis.columns[0]
df_tennis.keys()[0]

#Function to calculate the entropy of probaility of observations
# -p*log2*p

def entropy(probs):  
    import math
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

#Function to calulate the entropy of the given Data Sets/List with respect to target attributes
def entropy_of_list(a_list):  
    from collections import Counter
    cnt = Counter(x for x in a_list)   # Counter calculates the propotion of class
    num_instances = len(a_list)*1.0   # = 14
    print("\n Number of Instances of the Current Sub Class is {0}:".format(num_instances ))
    probs = [x / num_instances for x in cnt.values()]  # x means no of YES/NO
    print("\n Classes:",min(cnt),max(cnt))
    print(" \n Probabilities of Class {0} is {1}:".format(min(cnt),min(probs)))
    print(" \n Probabilities of Class {0} is {1}:".format(max(cnt),max(probs)))
    return entropy(probs) # Call Entropy :
   
    # The initial entropy of the YES/NO attribute for our dataset.
print("\n  INPUT DATA SET FOR ENTROPY CALCULATION:\n", df_tennis['PlayTennis'])

total_entropy = entropy_of_list(df_tennis['PlayTennis'])
print("\n Total Entropy of PlayTennis Data Set:",total_entropy)

def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    print("Information Gain Calculation of ",split_attribute_name)
    '''
    Takes a DataFrame of attributes, and quantifies the entropy of a target
    attribute after performing a split along the values of another attribute.
    '''
    # Split Data by Possible Vals of Attribute:
    df_split = df.groupby(split_attribute_name)
    for name,group in df_split:
            print("Name:\n",name)
            print("Group:\n",group)
    # Calculate Entropy for Target Attribute, as well as
    # Proportion of Obs in Each Data-Split
    nobs = len(df.index) * 1.0
    print("NOBS",nobs)
    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]
    print([target_attribute_name])
    print(" Entropy List ",entropy_of_list)
    print("DFAGGENT",df_agg_ent)
    df_agg_ent.columns = ['Entropy', 'PropObservations']
    if trace: # helps understand what fxn is doing:
        print(df_agg_ent)
    
    # Calculate Information Gain:
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy


print('Info-gain for Outlook is :'+str( information_gain(df_tennis, 'Outlook', 'PlayTennis')),"\n")
print('\n Info-gain for Humidity is: ' + str( information_gain(df_tennis, 'Humidity', 'PlayTennis')),"\n")
print('\n Info-gain for Wind is:' + str( information_gain(df_tennis, 'Wind', 'PlayTennis')),"\n")
print('\n Info-gain for Temperature is:' + str( information_gain(df_tennis, 'Temperature','PlayTennis')),"\n")

def id3(df, target_attribute_name, attribute_names, default_class=None):
    
    ## Tally target attribute:
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])# class of YES /NO
    
    ## First check: Is this split of the dataset homogeneous?
    if len(cnt) == 1:
        return next(iter(cnt))  # next input data set, or raises StopIteration when EOF is hit.
    
    ## Second check: Is this split of the dataset empty?
    # if yes, return a default value
    elif df.empty or (not attribute_names):
        return default_class  # Return None for Empty Data Set
    
    ## Otherwise: This dataset is ready to be devied up!
    else:
        # Get Default Value for next recursive call of this function:
        default_class = max(cnt.keys()) #No of YES and NO Class
        # Compute the Information Gain of the attributes:
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] #
        index_of_max = gainz.index(max(gainz)) # Index of Best Attribute
        # Choose Best Attribute to split on:
        best_attr = attribute_names[index_of_max]
        
        # Create an empty tree, to be populated in a moment
        tree = {best_attr:{}} # Iniiate the tree with best attribute as a node 
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        
        # Split dataset
        # On each split, recursively call this algorithm.
        # populate the empty tree with subtrees, which
        # are the result of the recursive call
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,
                        target_attribute_name,
                        remaining_attribute_names,
                        default_class)
            tree[best_attr][attr_val] = subtree
        return tree
# Get Predictor Names (all but 'class')
attribute_names = list(df_tennis.columns)
print("List of Attributes:", attribute_names) 
attribute_names.remove('PlayTennis') #Remove the class attribute 
print("Predicting Attributes:", attribute_names)
# Run Algorithm:
from pprint import pprint
tree = id3(df_tennis,'PlayTennis',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n")
#print(tree)
pprint(tree)
attribute = next(iter(tree))
print("Best Attribute :\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())

def classify(instance, tree, default=None): # Instance of Play Tennis with Predicted 
    
    print("Instance:",instance)
    attribute = next(iter(tree)) # Outlook/Humidity/Wind       
    print("Attribute:",attribute) # [Key /Attribute Both are same ]
   
    # print("Insance of Attribute :",instance[attribute],attribute)
    if instance[attribute] in tree[attribute].keys(): # Value of the attributs in  set of Tree keys  
        result = tree[attribute][instance[attribute]]
        print("Instance Attribute:",instance[attribute],"TreeKeys :",tree[attribute].keys())
        if isinstance(result, dict): # this is a tree, delve deeper
            return classify(instance, result)
        else:
            return result # this is a label
    else:
        return default
df_tennis['predicted'] = df_tennis.apply(classify, axis=1, args=(tree,'No') ) 
    # classify func allows for a default arg: when tree doesn't have answer for a particular
    # combitation of attribute-values, we can use 'no' as the default guess 

df_tennis[['PlayTennis', 'predicted']]
training_data = df_tennis.iloc[1:-4] # all but last four instances
test_data  = df_tennis.iloc[-4:] # just the last four
train_tree = id3(training_data, 'PlayTennis', attribute_names)

test_data['predicted2'] = test_data.apply( # <---- test_data source
                                          classify, 
                                          axis=1, 
                                          args=(train_tree,'Yes') ) # <---- train_data tree


print ('\n\n Accuracy is : ' + str( sum(test_data['PlayTennis']==test_data['predicted2'] ) / (1.0*len(test_data.index)) ))


 Given Play Tennis Data Set:

      Outlook Temperature Humidity    Wind PlayTennis
0      Sunny         Hot     High    Weak         No
1      Sunny         Hot     High  Strong         No
2   Overcast         Hot     High    Weak        Yes
3       Rain        Mild     High    Weak        Yes
4       Rain        Cool   Normal    Weak        Yes
5       Rain        Cool   Normal  Strong         No
6   Overcast        Cool   Normal  Strong        Yes
7      Sunny        Mild     High    Weak         No
8      Sunny        Cool   Normal    Weak        Yes
9       Rain        Mild   Normal    Weak        Yes
10     Sunny        Mild   Normal  Strong        Yes
11  Overcast        Mild     High  Strong        Yes
12  Overcast         Hot   Normal    Weak        Yes
13      Rain        Mild     High  Strong         No

  INPUT DATA SET FOR ENTROPY CALCULATION:
 0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes

 Classes: No Yes
 
 Probabilities of Class No is 0.3333333333333333:
 
 Probabilities of Class Yes is 0.6666666666666666:
['PlayTennis']
 Entropy List  <function entropy_of_list at 0x000001EABE031C10>
DFAGGENT              entropy_of_list  <lambda_0>
Temperature                             
Cool                1.000000         0.4
Mild                0.918296         0.6

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:
Information Gain Calculation of  Humidity
Name:
 High
Group:
    Outlook Temperature Humidity    Wind PlayTennis
3     Rain        Mild     High    Weak        Yes
13    Rain        Mild     High  Strong         No
Name:
 Normal
Group:
   Outlook Temperature Humidity    Wind PlayTennis
4    Rain        Cool   Normal    Weak        Yes
5    Rain        Cool   Normal  Strong         No
9    Rain        Mild   Normal    Weak        Yes
NOBS 5.0

 Number of Instances of the Cur

9    Rain        Mild   Normal  Weak        Yes       Yes
NOBS 9.0

 Number of Instances of the Current Sub Class is 4.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.25:
 
 Probabilities of Class Yes is 0.75:

 Number of Instances of the Current Sub Class is 2.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.5:
 
 Probabilities of Class Yes is 0.5:

 Number of Instances of the Current Sub Class is 3.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.3333333333333333:
 
 Probabilities of Class Yes is 0.6666666666666666:
['PlayTennis']
 Entropy List  <function entropy_of_list at 0x000001EABE031C10>
DFAGGENT              entropy_of_list  <lambda_0>
Temperature                             
Cool                0.811278    0.444444
Hot                 1.000000    0.222222
Mild                0.918296    0.333333

 Number of Instances of the Current Sub Class is 9.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.3333333333333333:
 
 Probabilities of Class Yes is 0.6666

<ipython-input-2-458e7791cc6f>:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted2'] = test_data.apply( # <---- test_data source


In [4]:
import numpy as np
import pandas as pd

data = pd.read_csv('tennis.csv')
print(data)
print('\n')
print(data.iloc[:,:-1])
print('\n')
print(data.iloc[:,-1])
def entropy(target):
    val,counts = np.unique(target,return_counts = True)
    ent = 0
    for i in range(len(val)):
        c = counts[i]/sum(counts)
        ent += -c*np.log2(c)
    return ent

def infoGain(data,features,target):
    te = entropy(data[target])
    val,counts = np.unique(data[features],return_counts = True)
    eg = 0
    for i in range(len(val)):
        c = counts[i]/sum(counts)
        eg += c*entropy(data[data[features] == val[i]][target])
    InfoGain = te-eg
    return InfoGain

def id3(data, features, target, pnode):
    
    t = np.unique(data[target])
    
    if len(t) == 1:
        return t[0]
    
    if len(features) == 0:
        return pnode
    
    pnode = t[np.argmax(t[1])]
    
    IG = [infoGain(data,f,target) for f in features]
    index = np.argmax(IG)
    
    col = features[index]
    tree = {col:{}}
    
    features = [f for f in features if f!=col]
    
    for val in np.unique(data[col]):
        sub_data = data[data[col]==val].dropna()
        subtree = id3(sub_data,features,target,pnode)
        tree[col][val] = subtree
    return tree

testData = data.sample(frac = 0.1)
data.drop(testData.index,inplace = True)

target = 'PlayTennis'
features = data.columns[data.columns!=target]

tree = id3(data,features,target,None)
print (tree, end='\n\n')

# test = testData.to_dict('records')[0]
test={'Outlook':'Sunny','Temperature':'Hot','Humidity':'High','Wind':'Weak','PlayTennis': 'Yes'}
print (test)
print(test, '=>', id3(test,features,target,None))

     Outlook Temperature Humidity    Wind PlayTennis
0      Sunny         Hot     High    Weak         No
1      Sunny         Hot     High  Strong         No
2   Overcast         Hot     High    Weak        Yes
3       Rain        Mild     High    Weak        Yes
4       Rain        Cool   Normal    Weak        Yes
5       Rain        Cool   Normal  Strong         No
6   Overcast        Cool   Normal  Strong        Yes
7      Sunny        Mild     High    Weak         No
8      Sunny        Cool   Normal    Weak        Yes
9       Rain        Mild   Normal    Weak        Yes
10     Sunny        Mild   Normal  Strong        Yes
11  Overcast        Mild     High  Strong        Yes
12  Overcast         Hot   Normal    Weak        Yes
13      Rain        Mild     High  Strong         No


     Outlook Temperature Humidity    Wind
0      Sunny         Hot     High    Weak
1      Sunny         Hot     High  Strong
2   Overcast         Hot     High    Weak
3       Rain        Mild     High  

In [5]:
def find_entropy(df):
    Class = df.keys()[-1] #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1] #To make the code generic, changing target variable class nam
    target_variables = df[Class].unique() #This gives all 'Yes' and 'No'
    variables = df[attribute].unique() #This gives different features in that attribute (like'Hot','Cold' in Temperature)
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
            fraction2 = den/len(df)
            entropy2 += -fraction2*entropy
    return abs(entropy2)
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:# Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]

def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)

def buildTree(df,tree=None):
    Class = df.keys()[-1] #To make the code generic, changing target variable class name #Here we build our decision tree #Get attribute with maximum information gain
    node = find_winner(df)#Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])#Create an empty dictionary to create tree
    if tree is None:
        tree={}
        tree[node] = {}#We make loop to construct a tree by calling this function recursively. #In this we check if the subset is pure and stops if it is pure.
    for value in attValue:
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['PlayTennis'],return_counts=True)
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]
        else:
            tree[node][value] = buildTree(subtable) #Calling the function recursively
    return tree
import pandas as pd
import numpy as np
eps = np.finfo(float).eps
from numpy import log2 as log
df = pd.read_csv('tennis.csv')
print("\n Given Play Tennis Data Set:\n\n",df)
tree= buildTree(df)

import pprint
print('The resultant decision tree is')
pprint.pprint(tree)
test={'Outlook':'Sunny','Temperature':'Hot','Humidity':'High','Wind':'Weak'}

def func(test, tree, default=None):
    attribute = next(iter(tree))
    print(attribute)
    if test[attribute] in tree[attribute].keys():
        print(tree[attribute].keys())
        print(test[attribute])
        result = tree[attribute][test[attribute]]
        if isinstance(result, dict):
            return func(test, result)
        else:
            return result
    else:
        return default
ans = func(test, tree)
print(ans)


 Given Play Tennis Data Set:

      Outlook Temperature Humidity    Wind PlayTennis
0      Sunny         Hot     High    Weak         No
1      Sunny         Hot     High  Strong         No
2   Overcast         Hot     High    Weak        Yes
3       Rain        Mild     High    Weak        Yes
4       Rain        Cool   Normal    Weak        Yes
5       Rain        Cool   Normal  Strong         No
6   Overcast        Cool   Normal  Strong        Yes
7      Sunny        Mild     High    Weak         No
8      Sunny        Cool   Normal    Weak        Yes
9       Rain        Mild   Normal    Weak        Yes
10     Sunny        Mild   Normal  Strong        Yes
11  Overcast        Mild     High  Strong        Yes
12  Overcast         Hot   Normal    Weak        Yes
13      Rain        Mild     High  Strong         No
The resultant decision tree is
{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'N

In [28]:
def entropy(df):
    Class=df.keys()[-1]
    val,count=np.unique(df[Class],return_counts=True)
    entropy=0
    for i in range(len(val)):
        fraction=count[i]/sum(count)
        entropy+=-fraction*np.log2(fraction)
    return entropy

def entropy_attr(df,attr):
    Class=df.keys()[-1]
    variables=df[attr].unique()
    target_attr=df[Class].unique()
    entr2=0
    for var in variables:
        ent=0
        for tar in target_attr:
            num=len(df[attr][df[attr]==var][df[Class]==tar])
            den=len(df[attr][df[attr]==var])
            frac2=num/(den+eps)
            entr2+=-frac2*log(frac2+eps)
            frac=den/(len(df))
            ent+=-entr2*frac
            
    return abs(ent)

def find_great(df):
    IG=[]
    for attr in df.keys()[:-1]:
        IG.append(entropy(df)-entropy_attr(df,attr))
        
    return df.keys()[np.argmax(IG)]

def get_subtable(df,node,value):
    return df[df[node]==value].reset_index(drop=True)

def buildTree(df,tree=None):
    Class=df.keys()[-1]
    node=find_great(df)
    attr=np.unique(df[node])
    if tree==None:
        tree={}
        tree[node]={}
    for value in attr:
        subtable=get_subtable(df,node,value)
        clValue,count=np.unique(subtable['PlayTennis'],return_counts=True)
        if len(count)==1:
            tree[node][value]=clValue[0]
        else:
            tree[node][value]=buildTree(subtable)
    return tree

import numpy as np
import pandas as pd
from numpy import log2 as log
df=pd.read_csv('tennis.csv')
eps=np.finfo(float).eps
print(df)
tree=buildTree(df)

import pprint
pprint.pprint(tree)

test={'Outlook':'Sunny','Temperature':'Hot','Humidity':'High','Wind':'Weak'}
def func(test,tree,default=None):
    attribute=next(iter(tree))
    print(attribute)
    if test[attribute] in tree[attribute].keys():
        print(tree[attribute].keys())
        print(test[attribute])
        result= tree[attribute][test[attribute]]
        if isinstance(result,dict):
            return func(test,result)
        else:
            return result
    else:
        return default


ans=func(test,tree)
print(ans)

     Outlook Temperature Humidity    Wind PlayTennis
0      Sunny        Cool     High    Weak         No
1      Sunny        Cool     High  Strong        Yes
2   Overcast         Hot     High    Weak        Yes
3       Rain        Mild     High    Weak        Yes
4       Rain        Mild   Normal    Weak        Yes
5       Rain        Cool   Normal  Strong         No
6   Overcast        Cool   Normal  Strong        Yes
7      Sunny        Mild     High    Weak         No
8      Sunny        Cool   Normal    Weak         No
9       Rain        Mild   Normal    Weak        Yes
10     Sunny        Mild     High  Strong        Yes
11  Overcast        Mild     High  Strong        Yes
12  Overcast         Hot   Normal    Weak        Yes
13      Rain        Mild     High  Strong        Yes
{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Temperature': {'Cool': 'No', 'Mild': 'Yes'}},
             'Sunny': {'Wind': {'Strong': 'Yes', 'Weak': 'No'}}}}
Outlook
dict_keys(['Overcast', 'Rain',